## Objective
Here, we inspect the denoiser performance. we use the stored prediction files to do that.

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
DEBUG=False
%run ./nb_core/root_dirs.ipynb
setup_syspath_disentangle(DEBUG)
%run ./nb_core/disentangle_imports.ipynb

In [ ]:
from disentangle.scripts.evaluate import * 
from disentangle.config_utils import get_configdir_from_saved_predictionfile, load_config
from disentangle.core.data_split_type import DataSplitType
from disentangle.core.tiff_reader import load_tiff
from disentangle.core.data_split_type import get_datasplit_tuples
import ml_collections



# data_dir = '/group/jug/ashesh/data/paper_stats/All_P128_G64_M50_Sk44/'
data_dir = '/group/jug/ashesh/data/paper_stats/All_P128_G64_M50_Sk32'
# data_dir = '/group/jug/ashesh/data/paper_stats/All_P128_G64_M50_Sk0'
denoiser_prediction_fname = "pred_disentangle_2402_D3-M23-S0-L0_11.tif"
channel_idx = 0

# get the prediction. 
pred = load_tiff(os.path.join(data_dir, denoiser_prediction_fname))
_, _ , test_idx = get_datasplit_tuples(0.1, 0.1, pred.shape[0], starting_test = False)
test_pred = pred[test_idx]
denoiser_configdir = get_configdir_from_saved_predictionfile(denoiser_prediction_fname)
print(denoiser_configdir)

# get the highres data
denoiser_config = load_config(denoiser_configdir)
denoiser_config = ml_collections.ConfigDict(denoiser_config)
if denoiser_config.data.data_type == DataType.BioSR_MRC:
    denoiser_input_dir = '/group/jug/ashesh/data/BioSR/'
elif denoiser_config.data.data_type == DataType.OptiMEM100_014:
    denoiser_input_dir = '/group/jug/ashesh/data/microscopy/OptiMEM100x014.tif'
elif denoiser_config.data.data_type == DataType.SeparateTiffData:
    denoiser_input_dir = '/group/jug/ashesh/data/ventura_gigascience/'
    denoiser_config.data.ch1_fname = denoiser_config.data.ch1_fname.replace('lowsnr', 'highsnr')
    denoiser_config.data.ch2_fname = denoiser_config.data.ch2_fname.replace('lowsnr', 'highsnr')
with denoiser_config.unlocked():
    highres_data = get_data_without_synthetic_noise(denoiser_input_dir, denoiser_config, DataSplitType.Test)

h, w = pred.shape[1:3]
highres_data = highres_data[:, :h, :w]
highres_data = highres_data[..., channel_idx].copy()

In [ ]:
_,ax = plt.subplots(figsize=(8,4),ncols=2)
ax[0].imshow(test_pred[-1])
ax[1].imshow(highres_data[-1,...])

In [ ]:
from disentangle.core.psnr import RangeInvariantPsnr
print(f'PSNR: {RangeInvariantPsnr(highres_data.astype(np.float32), test_pred.astype(np.float32)).mean().item():.2f}')

In [ ]:
## 33.76
## 33.20
##  32.90